In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import distance,lonlat
import math
import geopandas as gpd
from shapely.geometry import Point

In [17]:
fire_data=pd.read_csv("../data/output_data/output_fire_data/clustering_fire_data.csv")
fire_data

,latitude,longitude,frp
0,-36.979083,143.869362,9.0278
1,-37.395876,148.702175,61.1422
2,-36.360385,141.245716,10.0394
3,-37.326302,146.998582,17.1700
4,-37.521991,142.326735,13.3154
...,...,...,...
95,-35.033808,143.126835,1.2592
96,-38.101039,141.752253,6.5914
97,-34.821044,142.132761,3.4122
98,-38.242735,146.371048,21.8620


In [19]:
def obj_func(drone_cors):
    res=0
    for i in range(len(fire_data)):
        fire_point_monitor=0.0
        for j in range(len(drone_cors)):
            dis=distance(lonlat(drone_cors[j,0],drone_cors[j,1]),lonlat(float(fire_data.loc[i,['longitude']]),float(fire_data.loc[i,['latitude']]))).km
            if(dis<30):
                fire_point_monitor+=1/dis
        if fire_point_monitor==0:
            res+=float(fire_data.loc[i,['frp']])
        else:
            res+=(1/fire_point_monitor)*float(fire_data.loc[i,['frp']])
    return float(res)

In [21]:
def coverage(drone_cors):
    count=0
    for i in range(len(fire_data)):
        for j in range(len(drone_cors)):
            dis=distance(lonlat(drone_cors[j,0],drone_cors[j,1]),lonlat(float(fire_data.loc[i,['longitude']]),float(fire_data.loc[i,['latitude']]))).km
            if(dis<30):
                count+=1
                break
    return float(count)/float(len(fire_data))

In [ ]:
def random_blur(x_cor,y_cor):
    angle=np.random.rand()*math.pi*2
    return (x_cor+math.cos(angle)*blur_size,y_cor+math.sin(angle)*blur_size)

In [16]:
drone_number=80
rec_max_y=-33.65
rec_min_y=-39.57
rec_max_x=150.15
rec_min_x=140.67

drone_cors=np.random.random((drone_number,2))

In [18]:
file_path='../data/output_data/v_boundary.shp'
Victoria=gpd.read_file(file_path)

In [20]:
count=0
while True:
    if count==drone_number:
        break
    lon=np.random.rand()*(rec_max_y-rec_min_y)+rec_min_y
    lat=np.random.rand()*(rec_max_x-rec_min_x)+rec_min_x
    point=Point(lat,lon)
    if Victoria.geometry.contains(point).bool():
        drone_cors[count,0]=lat
        drone_cors[count,1]=lon
        count+=1

drone_cors

array([[143.18806613, -38.2414282 ],
       [149.76196433, -37.54593661],
       [141.51091108, -38.34791453],
       [147.27203505, -37.38525609],
       [142.67162701, -36.34373814],
       [143.43495013, -36.27197617],
       [143.57937397, -38.54377418],
       [147.95256452, -37.56389515],
       [146.07582256, -37.52773845],
       [144.22435687, -37.98628883],
       [147.00398428, -37.26193992],
       [143.26388828, -35.220588  ],
       [143.48402833, -37.72956159],
       [143.704148  , -38.17269568],
       [142.64643393, -35.07666526],
       [143.20641873, -38.67073056],
       [142.86027243, -35.58557678],
       [142.599274  , -37.85436476],
       [149.35187229, -37.75780116],
       [147.23241192, -37.8293174 ],
       [142.21220362, -36.36727553],
       [145.29957716, -37.03971902],
       [141.87738121, -36.50775362],
       [145.96017787, -37.45579475],
       [142.59499503, -37.55130069],
       [146.95633949, -36.5640046 ],
       [141.55457713, -35.19403105],
 

In [22]:
blur_size=0.01
start_temp=200
decay_constant=1.0001
end_temp=10
temp=start_temp
temp_old=start_temp
obj_value=10000000.0
best_result=[]
while temp>end_temp:
    temp=temp*decay_constant
    drone_cors_old=drone_cors.copy()
    obj_value_old=obj_value

    for i in range(len(drone_cors)):
        drone_cors[i,:]=random_blur(drone_cors[i,0],drone_cors[i,1])
    obj_value=coverage(drone_cors)
    # coverage_rate=coverage(drone_cors)
    print("obj_value",obj_value)
    # print("coverage",coverage_rate)
    if(obj_value>obj_value_old):
        best_result.append(obj_value)
        print("best_result",obj_value)

    else:
        obj_value=obj_value_old
        drone_cors=drone_cors_old.copy()
    

obj_value 0.55
obj_value 0.54
obj_value 0.54
obj_value 0.53
obj_value 0.54
obj_value 0.53
obj_value 0.53
obj_value 0.54
obj_value 0.54
obj_value 0.54
obj_value 0.53
obj_value 0.55
obj_value 0.54
obj_value 0.53
obj_value 0.54
obj_value 0.55
obj_value 0.53
obj_value 0.54
obj_value 0.55
obj_value 0.54
obj_value 0.53
obj_value 0.53
obj_value 0.55
obj_value 0.53
obj_value 0.55
obj_value 0.56
obj_value 0.54
obj_value 0.54
obj_value 0.54
obj_value 0.53
obj_value 0.54
obj_value 0.54
obj_value 0.53
obj_value 0.54
obj_value 0.54
obj_value 0.53
obj_value 0.53
obj_value 0.55
obj_value 0.54
obj_value 0.55
obj_value 0.55
obj_value 0.53
obj_value 0.55
obj_value 0.55
obj_value 0.56
obj_value 0.53
obj_value 0.54
obj_value 0.54
obj_value 0.56
obj_value 0.55
obj_value 0.55
obj_value 0.55
obj_value 0.54
obj_value 0.53
obj_value 0.54
obj_value 0.55
obj_value 0.53
obj_value 0.54
obj_value 0.54
obj_value 0.53
obj_value 0.54
obj_value 0.54
obj_value 0.55
obj_value 0.53
obj_value 0.53
obj_value 0.55
obj_value 

KeyboardInterrupt: 